In [3]:
from PIL import Image

In [4]:
import skimage.measure
def pooling_data(Iftrain, data, basepixel, pooling_size):
    
    #print pd.DataFrame(data)

    #大于某个像素值 ==> 255
    if Iftrain == True:
        if basepixel > 0:
            data[data> basepixel] =  255
            data[data<= basepixel] = 0
            
    #压缩矩阵操作,取平均值，你可以自己写函数
    pooled_data = skimage.measure.block_reduce(data,pooling_size,  np.mean)
    return pooled_data

In [5]:
import os


def save_to_image(Iftrain, pixels,basepixel,size, i):
    MINST = ""
    if Iftrain == True:
        MINST = "TRIAN_DATA"
    else:
        MINST = "TEST_DATA"
        
    image = Image.new('L',pixels.shape)
    columns = pixels.shape[0]
    rows = pixels.shape[1]
    
    for x in xrange(columns):
            for y in xrange(rows):
                    pix = pixels[y][x]
                    image.putpixel((y, x), int(pixels[y][x]))
    
    
    directory = "pooling/"+MINST+"/"+"basepixel_"+str(basepixel) +"/"+ str(columns)+"_"+str(rows)
    print directory+str(i)
    #print pd.DataFrame(pixels)
    
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    image.save(directory+"/"+str(i) + '.png')

In [6]:
def save_to_numpy(Iftrain, train,  label, basepixel, size):
    MINST = ""
    if Iftrain == True:
        MINST = "TRIAN_DATA"
    else:
        MINST = "TEST_DATA"
        
    directory = MINST+"/"+"basepixel_"+str(basepixel) +"/"+ str(size[0])+"_"+str(size[1])+"/"
    #print pd.DataFrame(pixels)
    
    if not os.path.exists(directory):
        os.makedirs(directory)
    ###################################################
    np.save(directory+'MINST_train.npy', train)
    test = np.load(directory+'MINST_train.npy')
    print np.all(train == test)  
    ###################################################
    np.save(directory+"MINST_label.npy", label)
    label_test = np.load(directory+'MINST_label.npy')
    print np.all(label == label_test)  
    ##################################################
    
    print train.shape
    with open(directory+'MINST_train.txt','wb') as f:
        for row in train:
             np.savetxt(f,np.transpose(row),delimiter=',',fmt = "%03d", footer='\n')
                
    with open(directory+'MINST_label.txt','wb') as f:
        for row in label:
             np.savetxt(f,row,delimiter=',',fmt = "%01d", footer='\n')            
    #x = np.loadtxt(directory+'MINST_train.txt',delimiter=',')
    #print np.all(x == train)

In [9]:
from PIL import Image
import struct
import numpy as np
import pandas as pd
import math

def deal_image(train_filename, label_filename, pooling_size_list, Debug_num = 0,Iftrain = True):
    
    ##############################################################################################
    train_file = open(train_filename, 'rb')
    buf = train_file.read()
    print "buf", len(buf)
    train_file.close()
    index = 0
    magic, images, rows, columns = struct.unpack_from('>IIII' , buf , index)
    print "magic", magic, "images", images, rows, columns
    index += struct.calcsize('>IIII')
    if Debug_num == 0:
        images_num = images
    else:
        images_num = Debug_num    
    ################################################################################################
    label_f = open(label_filename, 'rb')
    label_buf = label_f.read()
    label_index = 0
    struct.unpack_from('>II' , label_buf, label_index)
    label_index += struct.calcsize('>II')
    
    for size in pooling_size_list:
        for basepixel in range(0, 256, 25):
            
            pooled_size = (int(math.ceil(columns*1.0/size[0])) , int(math.ceil(rows*1.0/size[1])))
            print pooled_size
            
            
            ###################################################################
            np_to_save = np.zeros((images_num,  pooled_size[0], pooled_size[1]))
            label_to_save = np.zeros((images_num,1, 10))
            ###################################################################
            index = 0
            index += struct.calcsize('>IIII')
            ##############################
            ##############################
            label_index = 0
            label_index += struct.calcsize('>II')
            ##############################
            count_image = 0
            for i in xrange(images_num):
                # for debug
                normal_np = np.zeros((columns, rows))
                label_np = np.zeros(10)
                label_number = (struct.unpack_from('>B', label_buf, label_index)[0])
                label_index += struct.calcsize('>B')
                
                for x in xrange(columns):
                    for y in xrange(rows):
                        normal_np[y][x] = int(struct.unpack_from('>B', buf, index)[0])
                        index += struct.calcsize('>B')

 
                label_np[label_number] = 1

                #print normal_np.shape
                print np_to_save[i].shape
                pooled_data = pooling_data(Iftrain, normal_np, basepixel, size)
                ###############################################################
                np_to_save[i] =  pooled_data
                label_to_save[i] =label_np
                #print np_to_save[i]
                ################################################################
                save_to_image(Iftrain, pooled_data,basepixel,size, i)
               
            
            save_to_numpy(Iftrain, np_to_save, label_to_save, basepixel, pooled_size) 

In [10]:
#设置要池化的数据块大小
pooling_size_list = [(1,1), (2,2)]
#if Debug_num == 0, then save all images, Debug_num >0, save Debug_num images
deal_image("train-images.idx3-ubyte", "train-labels.idx1-ubyte",pooling_size_list,Debug_num=30,Iftrain=True)
deal_image("t10k-images.idx3-ubyte","t10k-labels.idx1-ubyte", pooling_size_list,Debug_num=30,Iftrain=False)


buf 47040016
magic 2051 images 60000 28 28
(28, 28)
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_280
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_281
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_282
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_283
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_284
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_285
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_286
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_287
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_288
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_289
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_2810
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_2811
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_2812
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_2813
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_2814
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_2815
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_2816
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_2817
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_2818
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_281

In [64]:
from PIL import Image
import struct
import numpy as np
import pandas as pd
import math

def deal_image_5_8(train_filename, label_filename, pooling_size_list, Debug_num = 0,Iftrain = True):
    
    ##############################################################################################
    train_file = open(train_filename, 'rb')
    buf = train_file.read()
    print "buf", len(buf)
    train_file.close()
    index = 0
    magic, images, rows, columns = struct.unpack_from('>IIII' , buf , index)
    print "magic", magic, "images", images, rows, columns
    index += struct.calcsize('>IIII')
    if Debug_num == 0:
        images_num = images
    else:
        images_num = Debug_num    
    ################################################################################################
    label_f = open(label_filename, 'rb')
    label_buf = label_f.read()
    label_index = 0
    struct.unpack_from('>II' , label_buf, label_index)
    label_index += struct.calcsize('>II')
    
    for size in pooling_size_list:
        for basepixel in range(0, 256, 25):
            
            pooled_size = (int(math.ceil(columns*1.0/size[0])) , int(math.ceil(rows*1.0/size[1])))
            print pooled_size
            
            
            ###################################################################
            np_to_save = np.zeros((20,  pooled_size[0], pooled_size[1]))
            label_to_save = np.zeros((20, 1, 10))
            ###################################################################
            index = 0
            index += struct.calcsize('>IIII')
            ##############################
            ##############################
            label_index = 0
            label_index += struct.calcsize('>II')
            ##############################
            count_image5  = 0
            count_image8  = 0
            count_image = 0
            for i in xrange(images_num):
                # for debug
                normal_np = np.zeros((columns, rows))
                label_np = np.zeros(10)
                label_number = (struct.unpack_from('>B', label_buf, label_index)[0])
                label_index += struct.calcsize('>B')
                for x in xrange(columns):
                    for y in xrange(rows):
                        normal_np[y][x] = int(struct.unpack_from('>B', buf, index)[0])
                        index += struct.calcsize('>B')

                
                label_np[label_number] = 1

                if label_number == 5 or label_number == 8:
                    
                        
            
                    
                    pooled_data = pooling_data(Iftrain, normal_np, basepixel, size)
                    ###############################################################
                    print np_to_save.shape
                    print pooled_data.shape
                    np_to_save[count_image] = pooled_data
                    label_to_save[count_image] = label_np
                    
                    
                    #print np_to_save[i]
                    ################################################################
                    save_to_image(Iftrain, pooled_data,basepixel,size, count_image)
                    
                    if label_number == 5:
                        if count_image5 >= 10:
                            continue 
                        count_image5 += 1
                        
                    if label_number == 8:
                        if count_image8>= 10:
                            continue 
                        count_image8 += 1
                        
                    count_image +=1 

                if count_image5 == 10 and count_image8 ==10:
                    print "fdsfa"
                    break
                
            
            save_to_numpy(Iftrain, np_to_save, label_to_save, basepixel, pooled_size) 

In [67]:
#设置要池化的数据块大小
pooling_size_list = [(1,1), (2,2)]
#if Debug_num == 0, then save all images, Debug_num >0, save Debug_num images
deal_image_5_8("train-images.idx3-ubyte", "train-labels.idx1-ubyte",pooling_size_list,Debug_num=600,Iftrain=True)
deal_image_5_8("t10k-images.idx3-ubyte","t10k-labels.idx1-ubyte", pooling_size_list,Debug_num=600,Iftrain=False)

buf 47040016
magic 2051 images 60000 28 28
(28, 28)
(20, 28, 28)
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_280
(20, 28, 28)
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_281
(20, 28, 28)
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_282
(20, 28, 28)
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_283
(20, 28, 28)
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_284
(20, 28, 28)
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_285
(20, 28, 28)
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_286
(20, 28, 28)
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_287
(20, 28, 28)
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_288
(20, 28, 28)
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_289
(20, 28, 28)
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_2810
(20, 28, 28)
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_2811
(20, 28, 28)
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_2812
(20, 28, 28)
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_2813
(20, 28, 28)
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_2814
(20, 28, 28)
(28, 28)
pooling/TRIAN_DATA/ba

In [ ]:
import numpy as np
from PIL import Image

# Creates a random image 100*100 pixels
mat = np.random.random((100,100))

# Creates PIL image
img = Image.fromarray(mat, 'L')
img.show() 

In [85]:
label_index = 0
label_index += struct.calcsize('>9B')
buf2 = open("train-labels.idx1-ubyte", "rb").read()
print struct.unpack_from('>B', buf2, 8)


(5,)


In [38]:
np.zeros(10)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [25]:
a = np.array([[1,3,4],[1,2,3],[1,2,1],[4,5,7]])
b = np.array([10,20,30])
a = np.vstack((a, b))

In [26]:
print b.shape

(3,)


In [27]:
print a.shape

(5, 3)
